In [1]:
# %pip install transformers datasets sentence-transformers faiss-cpu torch tqdm langchain langchain-community streamlit python-dotenv -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import requests
import re
import faiss
import pandas as pd
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from tqdm import tqdm_notebook
from tqdm.notebook import tqdm
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import pipeline
from datasets import load_dataset

from dotenv import load_dotenv
import os

load_dotenv()

API_TOKEN = os.getenv('API_TOKEN')
API_URL = 'https://api-inference.huggingface.co/models/mistralai/Mixtral-8x7B-Instruct-v0.1'

In [83]:
dataset = load_dataset('wikimedia/wikipedia', '20231101.ru', split='train')[:500]

README.md:   0%|          | 0.00/131k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

train-00000-of-00021.parquet:   0%|          | 0.00/581M [00:00<?, ?B/s]

train-00001-of-00021.parquet:   0%|          | 0.00/302M [00:00<?, ?B/s]

train-00002-of-00021.parquet:   0%|          | 0.00/250M [00:00<?, ?B/s]

...

train-00018-of-00021.parquet:   0%|          | 0.00/210M [00:00<?, ?B/s]

train-00019-of-00021.parquet:   0%|          | 0.00/187M [00:00<?, ?B/s]

train-00020-of-00021.parquet:   0%|          | 0.00/182M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1945063 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/21 [00:00<?, ?it/s]

In [ ]:
dataset['text'][1]

'Росси́я, или Росси́йская Федера́ция (),\xa0— государство в Восточной Европе и Северной Азии. Россия\xa0— крупнейшее государство в мире, её территория в международно признанных границах составляет  км². Население страны в тех же границах, но включая территорию украинского Крыма, аннексия Россией которого не получила международного признания, составляет  чел. (; 9-е место в мире).\n\nСтолица\xa0— Москва. Государственный язык на всей территории страны\xa0— русский, в ряде регионов России также установлены свои государственные и официальные языки. Денежная единица\xa0— российский рубль.\n\nРоссия\xa0— многонациональное государство с широким этнокультурным многообразием. Согласно результатам переписей населения России 2010\xa0года, а также аннексированных Россией украинских Крыма и Севастополя 2014\xa0года, в стране живут представители свыше 190 национальностей, среди которых русские составляют свыше 80\xa0%, а русским языком владеют свыше 99,4\xa0% россиян. Бо́льшая часть населения (около

### Предобработка документов

In [3]:
def chunk_text(data):
    # разбиение текста на чанки
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,  # Макс. размер чанка
        chunk_overlap=200,
        separators=["\n\n", "\n", "", ""]  # разделители для разбиения
    )

    documents = []
    # Разбиваем текст на чанки
    for i, item in enumerate(tqdm(data)):
        splits = text_splitter.split_text(item["text"])
        for j, split in enumerate(splits):  # Проходим по всем чанкам
            documents.append( # Создаем объект Document
                Document(
                    page_content=split,
                    metadata={"source": f"doc_{i}_part_{j}"}  # Метаданные: документ и номер фрагмента
                )
            )
    return documents

In [4]:
class DocumentProcessor:
    ''' Класс для обработки документов'''

    def __init__(
        self,
        chunk_size: int = 1000,
        chunk_overlap: int = 200,
        separators: list = ["\n\n", "\n", " ", ""],
        text_clean_patterns: list = [
            (r'<.*?>', ''),        # HTML tags
            (r'\.\.\.', ''),       # многоточие
            (r'…', ''),            # многоточие
            (r'[\u0301]', ''),     # диакритические значки
            (r'\s+', ' ')          # неск. пробелов
        ]
    ):
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=separators
        )
        self.text_clean_patterns = text_clean_patterns

    def preprocess_text(self, text: str) -> str: # Очистка текста
        text = text.strip()
        for pattern, replacement in self.text_clean_patterns:
            text = re.sub(pattern, replacement, text)
        return text

    def process_documents( # Обработка документов
        self,
        data: list[dict],
        content_key: str = "text",
        progress_bar: bool = True
    ) -> list[Document]:

        documents = []
        iter_data = tqdm(data, desc="Processing documents") if progress_bar else data

        for doc_id, item in enumerate(iter_data):
            try:
                raw_text = item[content_key]
            except KeyError:
                raise ValueError(f"Key '{content_key}' not found in document {doc_id}")

            cleaned_text = self.preprocess_text(raw_text)
            text_chunks = self.text_splitter.split_text(cleaned_text)

            for chunk_id, chunk in enumerate(text_chunks):
                documents.append(
                    Document(
                        page_content=chunk,
                        metadata={
                            "source": f"doc_{doc_id}_part_{chunk_id}",
                            "original_doc_id": doc_id
                        }
                    )
                )

        return documents

In [5]:
processor = DocumentProcessor(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", "."]
)

In [97]:
data = [{'text': text} for text in dataset['text']]

In [98]:
processed_docs = processor.process_documents(
    data=data,
    content_key='text',
    progress_bar=True
    )

Processing documents:   0%|          | 0/500 [00:00<?, ?it/s]

In [101]:
docs = chunk_text(data)

  0%|          | 0/500 [00:00<?, ?it/s]

In [10]:
embeddings = HuggingFaceEmbeddings(model_name = 'all-MiniLM-L6-v2') # Загружаем модель для создания эмбеддингов

<ipython-input-10-a1d670a2a161>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = 'all-MiniLM-L6-v2') # Загружаем модель
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Создание векторного хранилища FAISS

In [11]:
def create_vector_store(documents, embeddings, vector_store_path="faiss_index"):
    '''
    Функция для создания векторного хранилища FAISS.

    Args:
        documents: Список документов для добавления в хранилище.
        embeddings: Модель Hugging Face для преобразования текста в эмбеддинги.
        vector_store_path: Путь для сохранения векторного хранилища на диске.

    Returns:
        vector_store: Объект векторного хранилища FAISS.
    '''

    vector_store = None # Инициализируем хранилище

    with tqdm(total=len(documents), desc="Ingesting documents") as pbar:
        for doc in documents:
            if vector_store:
                # Если хранилище уже создано, добавляем документ
                vector_store.add_documents([doc])
            else:
                # Если хранилище ещё не создано, создаем его и добавляем первый документ
                vector_store = FAISS.from_documents([doc], embeddings)

            pbar.update(1)  # Обновляем прогресс

    # Сохраняем созданное векторное хранилище в локальный файл
    vector_store.save_local(vector_store_path)

    return vector_store

In [102]:
vector_store = create_vector_store(docs, embeddings)

Ingesting documents:   0%|          | 0/14171 [00:00<?, ?it/s]

In [103]:
vector_store

In [356]:
def create_prompt(query, relevant_docs):
    '''Функция, создающая промпт для модели-генератора'''

    # Очистка документов от технических артефактов
    cleaned_docs = [
        re.sub(r'doc_\d+', '', doc.page_content).strip()
        for doc in relevant_docs
    ]
    docs_summary = "\n".join(cleaned_docs)[:2000]  # Ограничение контекста

    prompt = f"""
    <s>[INST] <<SYS>>
    Язык ответа: Русский.
    Ты ассистент, отвечающий исключительно на основе предоставленных данных на русском языке. Строго соблюдай правила:

    1. Если ответа нет в данных → "Информация не найдена"
    2. Только 1 предложение (50-100 символов) на русском языке.
    3. Запрещено:
      - Упоминать источники/документы
      - Технические термины (doc_123)
      - Маркированные списки
      - Любые разделы кроме ответа
      - Выдумывать ответы
    <</SYS>>

    ### Контекст ###
    {docs_summary}

    ### Примеры ###
    Вопрос: Какая площадь России?
    Ответ: 17.1 млн км².

    Вопрос: Сколько сейчас времени?
    Ответ: Информация не найдена.

    ### Задача ###
    Вопрос: {query}
    Ответ: [/INST]</s>
    """
    return prompt

In [15]:
vector_store.as_retriever # Используем векторное хранилище в качестве ретривера

<bound method VectorStore.as_retriever of <langchain_community.vectorstores.faiss.FAISS object at 0x79e97521f1d0>>

### Генерация ответов

In [236]:
def get_response(query, vector_store, api_url, api_token):
    '''
    Функция для получения ответа от языковой модели через API Hugging Face с использованием релевантных
    документов из хранилища FAISS.

    Args:
        student_query (str): Запрос пользователя
        vector_store (FAISS): Векторное хранилище FAISS
        api_url (str): URL API Hugging Face Inference Endpoint
        api_token (str): Токен API Hugging Face

    Returns:
        str: Ответ, сгенерированный языковой моделью.

    Raises:
        ValueError: Если API возвращает ошибку.
    '''

    retriever = vector_store.as_retriever(search_kwargs={"k": 3})  # Извлекаем топ-3 релевантных документа
    relevant_docs = retriever.invoke(query)

    # Формируем промт
    prompt = create_prompt(query, relevant_docs)

    # Отправляем запрос к API Hugging Face
    headers = {
        "Authorization": f"Bearer {api_token}"
    }
    payload = {
        "inputs": prompt,
        "parameters": {
            "max_length": 2000,
            "temperature": 0.7,
            "num_return_sequences": 1,
        }
    }

    response = requests.post(api_url, headers=headers, json=payload)

    try:
        # Обработка успешного ответа
        response.raise_for_status() # Проверяет, был ли запрос успешным (код ответа 2xx)
        output = response.json() # преобразуем ответ из JSON в Python-объект
        return output[0]["generated_text"][len(prompt):] # Извлекаем сгенерированный текст из ответа

    except requests.exceptions.HTTPError as err:
        # Обработка ошибок
        if err.response.status_code == 500:  # 500 errors
            print(f"Возникла ошибка: {err}")
            return "Попробуйте позже"
        else:
            raise  # Повторно генерирует исключение, чтобы остановить дальнейшее выполнение функции

    if response.status_code == 200: # Дополнительная проверка успешного ответа
        output = response.json()
        return output[0]["generated_text"][len(prompt):]  # Извлекаем сгенерированный текст, отсекая промт
    else:
        raise ValueError(f"Ошибка API: {response.status_code}, {response.text}") # генерируется исключение ValueError

    if err.response.status_code == 500:  # Check for 500 errors
        print(f"HTTP error occurred: {err}")
        return "Попробуйте позже"

Тестирование RAG-системы

In [321]:
query = 'Столица России?' # информация содержится в наборе документов
response = get_response(query, vector_store, API_URL, API_TOKEN)
print(response.strip())

Ответ: Москва является столицей России.


In [128]:
query = 'Кто такой Лев Никифоров?' # информация из статьи, которая не попала в выборку
response = get_response(query, vector_store, API_URL, API_TOKEN)
print(response.strip())

Информация не найдена.


In [122]:
query = 'Как сделать rag-систему?' # пример нерелевантного запроса
response = get_response(query, vector_store, API_URL, API_TOKEN)
print(response.strip())

Информация не найдена. К сожалению, я не нашёл данных о способах создания RAG-системы.


### Сравнение с эталонными ответами и оценка метрик

In [315]:
df = pd.read_csv('QA_RAG.csv') # загружаем df с эталонными ответами
df.head()

,questions,ref_answers
0,Столица Литвы?,Столица Литвы — Вильнюс.
1,Что такое лингвистика?,"Лингвистика — это наука, изучающая язык."
2,В каком году основана Российская империя?,Российская империя основана в 1721 году.
3,Какое крупнейшее государство в мире?,Россия — крупнейшее государство в мире.
4,Из какого отряда слоны?,Слоны относятся к отряду хоботных.


In [256]:
# Создаем пустой список для хранения ответов
answers = []

# Итерируемся по df
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    query = row['questions'] # Извлекаем вопрос из текущей строки

    # Вызываем get_response для получения ответа на вопрос
    try:
        response = get_response(query, vector_store, API_URL, API_TOKEN)
    except Exception as e:
        print(f"Ошибка при обработке вопроса {query}: {e}")
        response = "None"

    answers.append(response)

  0%|          | 0/50 [00:00<?, ?it/s]

Возникла ошибка: 500 Server Error: Internal Server Error for url: https://api-inference.huggingface.co/models/mistralai/Mixtral-8x7B-Instruct-v0.1


In [316]:
df['gen_answers'] = pd.Series(answers)

In [317]:
df.head()

,questions,ref_answers,gen_answers
0,Столица Литвы?,Столица Литвы — Вильнюс.,Столица Литвы - Вильнюс.
1,Что такое лингвистика?,"Лингвистика — это наука, изучающая язык.","Лингвистика - наука о языке, изучающая строени..."
2,В каком году основана Российская империя?,Российская империя основана в 1721 году.,1721 год.
3,Какое крупнейшее государство в мире?,Россия — крупнейшее государство в мире.,Самая крупная по площади страна в мире — Россия
4,Из какого отряда слоны?,Слоны относятся к отряду хоботных.,Слоны относятся к отряду хоботных.


Оценка косинусной близости

In [328]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

def calculate_similarity(ref_answer, gen_answer):
    embeddings = model.encode([ref_answer, gen_answer]) # генерируем эмбеддинги ответов системы
    similarity = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0] # вычисляем косинусную близость
    return similarity

# вычисляем косинусную близость для каждой пары ответов
df['similarity'] = df.apply(lambda row: calculate_similarity(row['ref_answers'], row['gen_answers']), axis=1)

average_similarity = df['similarity'].mean() # среднее значение
print(f"Средняя косинусная близость: {average_similarity:.2f}")

Средняя косинусная близость: 0.78


Средняя косинусная близость 0.78 указывает на высокую степень семантической схожести, т.е. объекты имеют значительное смысловое пересечение.

Оценка BERT-Score

In [355]:
from bert_score import score

def calculate_bert_score(ref_answer, gen_answer):
  P, R, F1 = score(gen_answer, ref_answer, lang="ru", verbose=True)
  return P.mean(), R.mean(), F1.mean()

ref_answer = df['ref_answers'].tolist()
gen_answer = df['gen_answers'].tolist()

precision, recall, f1 = calculate_bert_score(ref_answer, gen_answer)

print(f"BERT Score - Precision: {precision:.2f}")
print(f"BERT Score - Recall: {recall:.2f}")
print(f"BERT Score - F1: {f1:.2f}")


calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 11.14 seconds, 4.49 sentences/sec
BERT Score - Precision: 0.79
BERT Score - Recall: 0.81
BERT Score - F1: 0.80


**Precision: 0.79** — модель сохраняет смысловую точность — 79% ответа соответствует эталону.

**Recall: 0.81** — ответ покрывает 81% важных деталей эталона.

**F1: 0.80** — модель демонстрирует хороший баланс между точностью и полнотой.